In [19]:
import sqlite3
import json
from langchain.agents import Tool, create_react_agent, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain.tools import tool
from langchain_openai import AzureChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain import hub

In [2]:
OPENAI_DEPLOYMENT_ENDPOINT = "https://az-openai-document-question-answer-service.openai.azure.com/" 
OPENAI_API_KEY = "5d24331966b648738e5003caad552df8" 
OPENAI_API_VERSION = "2023-05-15"

OPENAI_DEPLOYMENT_NAME = "az-gpt_35_model"
OPENAI_MODEL_NAME="gpt-3.5-turbo"

OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME = "az-embedding_model" 
OPENAI_ADA_EMBEDDING_MODEL_NAME = "text-embedding-ada-002"

encoding_name = "cl100k_base"
# OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
# CHROMA_PATH = "c:\\Users\\SujaySunilNagvekar\\VM\\GEN AI\\SERFF\\vector_db\\neil_db"
# data_folder = "C:\\Users\\SujaySunilNagvekar\\VM\\GEN AI\\SERFF\\new_files"
csv_file = "C:\\Users\\SujaySunilNagvekar\\VM\\GEN AI\\Parker\\vm-GenAI_BI\\Text to SQL_V1\\Nurse_notes_parker.csv"
### Cohere API key 

In [3]:
db_file = "nurse_notes.db"

In [4]:
llm = AzureChatOpenAI(
                        temperature=0.1,
                        deployment_name=OPENAI_DEPLOYMENT_NAME,
                        model_name=OPENAI_MODEL_NAME,
                        azure_endpoint=OPENAI_DEPLOYMENT_ENDPOINT,
                        openai_api_version=OPENAI_API_VERSION,
                        openai_api_key=OPENAI_API_KEY            
                    )

# Step 2: Connect to SQLite Database
def connect_db(db_file):
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    return conn, cursor

In [5]:
def extract_schema(cursor, table_name):
    cursor.execute(f"PRAGMA table_info({table_name})")
    columns = cursor.fetchall()
    schema = []
    for column in columns:
        schema.append({
            "Column Name": column[1],
            "Data Type": column[2],
            "Nullable": 'YES' if column[3] == 0 else 'NO',
            "Primary Key": 'YES' if column[5] > 0 else 'NO'
        })
    return schema

In [6]:
def extract_metadata(cursor, table_name):
    cursor.execute(f"SELECT * FROM {table_name} LIMIT 5")
    rows = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]

    metadata = []
    for col in columns:
        example_values = []
        for row in rows:
            idx = [i for i, desc in enumerate(cursor.description) if desc[0] == col]
            if idx:  # Check if index exists
                example_values.append(row[idx[0]] if row[idx[0]] is not None else "NULL")

        metadata.append({
            "Column Name": col,
            "Examples": example_values[:3]
        })
    return metadata

In [7]:
def process_all_tables(db_file):
    conn, cursor = connect_db(db_file)
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    schema_info = {}
    metadata_info = {}

    for table in tables:
        table_name = table[0]
        schema_info[table_name] = extract_schema(cursor, table_name)
        metadata_info[table_name] = extract_metadata(cursor, table_name)

    conn.close()
    return schema_info, metadata_info

In [8]:
def save_schema_metadata(schema, metadata):
    with open('schema.json', 'w') as schema_file:
        json.dump(schema, schema_file, indent=4)

    with open('metadata.json', 'w') as metadata_file:
        json.dump(metadata, metadata_file, indent=4)

In [9]:
# Example Tool: SQL Query Executor
def query_sqlite(query: str) -> str:
    """Executes SQL query and returns results."""
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]
        conn.close()
        return str([dict(zip(columns, row)) for row in result])
    except sqlite3.Error as e:
        conn.close()
        return f"SQL error: {e}"

# Define Tools for the Agent
tools = [
    Tool(
        name="Query SQLite",
        func=query_sqlite,
        description="Executes SQL queries and retrieves results."
    )
]

In [112]:
import pandas as pd

def generate_table_metadata():
    """
    Generates metadata string for the table, including column names, types, and descriptions.

    Returns:
        str: Formatted metadata string for the prompt.
    """
    # Define table metadata
    table_metadata = pd.DataFrame({
        "Column_Name": [
            "Primary_key", "First_Name", "Last_Name", "Age", "Gender",
            "Marital_Status", "Race", "Day", "Shift", "Nurse_name",
            "Clinical_Notes", "Non_clinical_notes"
        ],
        "Column_Type": [
            "INTEGER", "varchar", "varchar", "INTEGER", "varchar",
            "varchar", "varchar", "varchar", "varchar", "varchar",
            "varchar", "varchar"
        ],
        "Description": [
            "Unique identifier for each record",
            "First name of the senior resident",
            "Last name of the senior resident",
            "Age of the senior resident",
            "Gender of the senior resident (Male/Female)",
            "Marital status of the resident (Single/Married/Widowed/Divorced)",
            "Race of the resident",
            "Day of the week the note was recorded",
            "Shift during which the note was recorded (Morning/Night)",
            "Full Name of the nurse who recorded the note",
            "Detailed clinical notes about the resident's incident or medical event/ emergency",
            "Non-clinical notes such as personal preferences or complaints"
        ]
    })

    # Format metadata as a string
    formatted_metadata = table_metadata.apply(
        lambda row: f"{row['Column_Name']} ({row['Column_Type']}): {row['Description']}".replace("{", "{{").replace("}", "}}"),
        axis=1
    ).tolist()

    # Combine metadata into a single string
    metadata_string = "\n".join(formatted_metadata)

    return metadata_string


In [133]:
schema, _  = process_all_tables(db_file)
table_name = "nurse_notes"
table_metadata = generate_table_metadata()


In [136]:
import pandas as pd

def generate_metadata_prompt(table_metadata, table_name):
    """
    Combines schema, metadata, and table name into a single formatted string
    for inclusion in the prompt template.

    Args:
        schema (str): The schema string generated by process_all_tables().
        table_metadata (pd.DataFrame): Metadata DataFrame with columns like Name, Type, Description.
        table_name (str): Name of the table being queried.

    Returns:
        str: Formatted metadata string to embed in the prompt.
    """



    # Combine Everything
    metadata_prompt = f"""
Table Name: {table_name}


Metadata for the Table:
{table_metadata}
"""

    return metadata_prompt


In [137]:
metadata_string = generate_metadata_prompt(table_metadata, table_name)

# Print Result (For Debugging)
print(metadata_string)


Table Name: nurse_notes


Metadata for the Table:
Primary_key (INTEGER): Unique identifier for each record
First_Name (varchar): First name of the senior resident
Last_Name (varchar): Last name of the senior resident
Age (INTEGER): Age of the senior resident
Gender (varchar): Gender of the senior resident (Male/Female)
Marital_Status (varchar): Marital status of the resident (Single/Married/Widowed/Divorced)
Race (varchar): Race of the resident
Day (varchar): Day of the week the note was recorded
Shift (varchar): Shift during which the note was recorded (Morning/Night)
Nurse_name (varchar): Full Name of the nurse who recorded the note
Clinical_Notes (varchar): Detailed clinical notes about the resident's incident or medical event/ emergency
Non_clinical_notes (varchar): Non-clinical notes such as personal preferences or complaints



In [195]:
# Step 3: Memory for Chat History
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Step 4: Define the Prompt for the Agent

# prompt = hub.pull("hwchase17/react")

# Step 4: Define the Prompt for the Agent
prompt = PromptTemplate(
    input_variables=[
        'agent_scratchpad', 'input', 'tool_names', 'tools',  # FLAT variables
    ],
    template=f'''
You are an AI assistant equipped with tools for querying databases and reasoning step-by-step. 
Your job is to carefully analyze the input query, determine the best tool to use, EXECUTE the tool.

Metadata for the Table:
{metadata_string}


Use the following format:

Question: {{input}}
Thought: you should always think about what to do
Action: the action to take, should be one of [{{tool_names}}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

TOOLS:
{{tools}}

AVAILABLE TOOL NAMES:
{{tool_names}}

Begin!

Question: {{input}}
Thought:{{agent_scratchpad}}
'''
)


# Step 5: Create a Custom Output Parser
def clean_output(response):
    """
    Custom function to extract only the final answer from agent response.
    """
    # Check if response includes explanations or reasoning; extract final answer
    if "Therefore," in response:
        return response.split("Therefore,")[-1].strip()  # Extract text after 'Therefore'
    return response.strip()

output_parser = None  # No custom parser for now


# Step 6: Create the ReAct Agent
agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)

# Step 7: Wrap in Executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    memory=memory,
    return_intermediate_steps=False,  # Suppress reasoning steps
    output_parser=output_parser,
    handle_parsing_errors=True,  # Apply the custom output parser
    max_iterations=15,  # Increase iteration limit
    max_execution_time=30  # Add execution timeout
)

In [196]:
# prompt = hub.pull("hwchase17/react")

In [197]:
memory

ConversationBufferMemory(return_messages=True, memory_key='chat_history')

In [201]:
response = agent_executor.invoke({
    "input": "Which Resident complained about lack of acitivites"})

print(response['output'])




> Entering new AgentExecutor chain...
I need to query the nurse_notes table to find the resident who complained about lack of activities. I should look for non-clinical notes that mention this complaint.
Action: Query SQLite
Action Input: SELECT First_Name, Last_Name FROM nurse_notes WHERE Non_clinical_notes LIKE '%lack of activities%'[{'First_Name': 'Robert', 'Last_Name': 'Nash'}]The query returned one result, which means that Robert Nash is the only resident who complained about lack of activities.
Final Answer: Robert Nash complained about lack of activities.

> Finished chain.
Robert Nash complained about lack of activities.


In [202]:
print(query_sqlite("SELECT COUNT(*) FROM nurse_notes WHERE Nurse_name='Olivia'"))


[{'COUNT(*)': 0}]
